In [1]:
import pandas as pd

datadf = pd.read_csv("BTC-USD.csv")

In [2]:
datadf.head

<bound method NDFrame.head of            Date          Open          High           Low         Close  \
0    2023-06-18  26510.457031  26675.925781  26325.890625  26336.212891   
1    2023-06-19  26335.441406  26984.611328  26312.832031  26851.029297   
2    2023-06-20  26841.664063  28388.968750  26668.791016  28327.488281   
3    2023-06-21  28311.310547  30737.330078  28283.410156  30027.296875   
4    2023-06-22  29995.935547  30495.998047  29679.158203  29912.281250   
..          ...           ...           ...           ...           ...   
362  2024-06-14  66747.570313  67294.648438  65056.894531  66011.093750   
363  2024-06-15  66006.742188  66402.187500  65871.773438  66191.000000   
364  2024-06-16  66189.359375  66894.843750  66018.250000  66639.046875   
365  2024-06-17  66636.515625  67188.320313  65094.964844  66490.296875   
366  2024-06-18  66490.976563  66491.226563  64637.632813  65632.835938   

        Adj Close       Volume  
0    26336.212891   9565695129  
1  

In [5]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

import os

os.environ['GROQ_API_KEY'] = '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'

In [6]:
llm = ChatGroq(model_name="llama3-8b-8192", temperature=1, top_p=1, max_tokens=1024)
llm.invoke("what day is today?")

/home/nomad/Desktop/PlsHireMe/PHMvenv/lib/python3.11/site-packages/langchain_groq/chat_models.py:150: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  warnings.warn(


AIMessage(content="I'm an AI, I don't have have access to real-time information, including the current date and time. I'm a large language model, I was trained on a dataset that was frozen in time, and I don't have the ability to update my knowledge in real-time.\n\nHowever, if you want to know the current date and time, I can suggest some ways to find out:\n\n1. Check your device clock: You can check your phone, watch, or computer clock to see the current date and time.\n2. Ask a time-tracking website: You can visit websites like timeanddate.com or worldtimebuddy.com to find out the current date and time.\n3. Ask a virtual assistant: You can ask virtual assistants like Alexa, Google Assistant, or Siri to tell you the current date and time.\n\nI hope that helps!", response_metadata={'token_usage': {'completion_tokens': 172, 'prompt_tokens': 15, 'total_tokens': 187, 'completion_time': 0.141926812, 'prompt_time': 0.003583984, 'queue_time': None, 'total_time': 0.14551079600000003}, 'model

In [17]:
from datetime import datetime
from datetime import timedelta

FakeDate = datetime(2024, 6, 18)
FakeDateFormatted = FakeDate.strftime("%Y-%m-%d")
FakeDateFormatted

'2024-06-18'

In [16]:
Today = datetime.utcnow()
TodayFormatted = Today.strftime("%Y-%m-%d")
TodayFormatted

'2024-06-19'

In [18]:
FakeDateFormatted-timedelta(days=1)

TypeError: unsupported operand type(s) for -: 'str' and 'datetime.timedelta'

In [27]:
#get subset of data

startdate= FakeDate - timedelta(days=1)
#startdate= startdate.strftime("%Y-%m-%d")
enddate= FakeDate

startdate

datetime.datetime(2024, 6, 17, 0, 0)

In [31]:
ssdata = datadf
ssdata["Date"] = pd.to_datetime(ssdata["Date"])

subset = datadf[(datadf["Date"] >= pd.to_datetime(startdate)) & (datadf["Date"] <= pd.to_datetime(enddate))]

In [32]:
subset

,Date,Open,High,Low,Close,Adj Close,Volume
365,2024-06-17,66636.515625,67188.320313,65094.964844,66490.296875,66490.296875,30006354476
366,2024-06-18,66490.976563,66491.226563,64637.632813,65632.835938,65632.835938,36443811840


In [49]:
from langchain_community.utilities.sql_database import SQLDatabase

#db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [56]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///btcusd.db', echo=False)
ssdb = subset.to_sql(name='btcusd', con=engine)

In [60]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent_types import AgentType

agent = create_sql_agent(
    ChatGroq(model_name="llama3-8b-8192", temperature=1, model_kwargs={'top_p': 1}, max_tokens=1024),
    #db=db,
    db=SQLDatabase.from_uri('sqlite:///btcusd.db'),
    verbose=True,
    #agent_type="openai-tools"
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    #allow_dangerous_code=True
)

In [61]:
agent.invoke(
    #"You are given the following Columns in the Table: Date, Open, High, Low, Close.Which day has seen the highest opening price?"
    "which day has seen the highest opening price?"

)



> Entering new SQL Agent Executor chain...
Let's get started.

Thought: I need to retrieve the highest opening price for each day. I should start by getting a list of tables to query from.

Action: sql_db_list_tables
Action Input: btcusdThought: I now have the list of tables.

Action: sql_db_schema
Action Input: btcusd
CREATE TABLE btcusd (
	"index" BIGINT, 
	"Date" DATETIME, 
	"Open" FLOAT, 
	"High" FLOAT, 
	"Low" FLOAT, 
	"Close" FLOAT, 
	"Adj Close" FLOAT, 
	"Volume" BIGINT
)

/*
3 rows from btcusd table:
index	Date	Open	High	Low	Close	Adj Close	Volume
365	2024-06-17 00:00:00	66636.515625	67188.320313	65094.964844	66490.296875	66490.296875	30006354476
366	2024-06-18 00:00:00	66490.976563	66491.226563	64637.632813	65632.835938	65632.835938	36443811840
*/What a delightful challenge!

Thought: Now that I have the schema and sample rows, I need to write a query to retrieve the highest opening price for each day.

Thought: I will use the GROUP BY clause to group the data by the "Date" 

{'input': 'which day has seen the highest opening price?',
 'output': 'The day that has seen the highest opening price is 2024-06-17, with an opening price of 66636.515625.'}

In [62]:
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mpl_dates

plt.style.use('ggplot')

btcbis = btc.iloc[-100:]

# Extracting Data for plotting
ohlc = btcbis.loc[:, ['Date', 'Open', 'High', 'Low', 'Close']]
ohlc['Date'] = pd.to_datetime(ohlc['Date'])
ohlc['Date'] = ohlc['Date'].apply(mpl_dates.date2num)
ohlc = ohlc.astype(float)

# Creating Subplots
fig, ax = plt.subplots()

candlestick_ohlc(ax, ohlc.values, width=0.6, colorup='green', colordown='red', alpha=0.8)

# Setting labels & titles
ax.set_xlabel('Date')
ax.set_ylabel('Price')
ax.set_facecolor('white')
fig.suptitle('Daily Candlestick Chart')

# Formatting Date
date_format = mpl_dates.DateFormatter('%d-%m-%Y')
ax.xaxis.set_major_formatter(date_format)
fig.autofmt_xdate()

fig.tight_layout()
fig.set_size_inches(10,8)

plt.savefig("chartlast100candles.png")

/home/nomad/Desktop/PlsHireMe/PHMvenv/lib/python3.11/site-packages/mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


NameError: name 'btc' is not defined

In [ ]:
loader = CSVLoader(file_path="BTC-USD.csv", encoding="utf-8")
data = loader.load()


# Embed
vectorstore = Chroma.from_documents(documents=data, 
                                    embedding=HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2"))

retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatGroq(model_name="llama3-8b-8192", temperature=1, top_p=1, max_tokens=1024)

# Chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("What is the average of the Closes in the Dataset?")